# DivNoising - Training
This notebook contains an example on how to train a DivNoising VAE.  This requires having a noise model (model of the imaging noise) which can be either measured from calibration data or estimated from raw noisy images themselves. If you haven't done so, please first run 'Convallaria-CreateNoiseModel.ipynb', which will download the data and create a noise model. 

In [1]:
# We import all our dependencies.
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import Dataset, DataLoader
from torch.nn import init
import os
import glob
from tifffile import imread
from matplotlib import pyplot as plt

import sys
sys.path.append('../../')
from divnoising import dataLoader
from divnoising import utils
from divnoising import training_multi
from nets import model_multi
from divnoising import histNoiseModel
from divnoising.gaussianMixtureNoiseModel import GaussianMixtureNoiseModel

import urllib
import os
import zipfile

from tqdm import tqdm

In [2]:
device = torch.device("cuda:0")

### Specify ```path``` to load data
Your data should be stored in the directory indicated by ```path```.

In [3]:
# create a folder for our data
if not os.path.isdir('./data'):
    os.mkdir('data')

# check if data has been downloaded already
zipPath="data/BSD68_reproducibility.zip"
if not os.path.exists(zipPath):
    #download and unzip data
    data = urllib.request.urlretrieve('https://cloud.mpi-cbg.de/index.php/s/pbj89sV6n6SyM29/download', zipPath)
    with zipfile.ZipFile(zipPath, 'r') as zip_ref:
        zip_ref.extractall("data")

In [4]:
signal = np.load('data/BSD68_reproducibility_data/test/bsd68_groundtruth.npy', allow_pickle=True)
train_images_ = signal[:int(0.85*signal.shape[0])]
val_images_ = signal[int(0.85*signal.shape[0]):]

In [5]:
patch_size = 128
img_width = 321
img_height = 481
num_patches = int(float(img_width*img_height)/float(patch_size**2)*2)

x_train_crops = utils.extract_patches(train_images_, patch_size, num_patches)
x_val_crops = utils.extract_patches(val_images_, patch_size, num_patches)
train_images = x_train_crops
val_images = x_val_crops

  0%|          | 0/57 [00:00<?, ?it/s]/home/prakash/anaconda3/envs/FastAI/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass max_patches=18 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/prakash/anaconda3/envs/FastAI/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass max_patches=18 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/prakash/anaconda3/envs/FastAI/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass max_patches=18 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/prakash/anaconda3/envs/FastAI/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass max_patches=18 as keyword args. From version 0.25 passing these as positional arguments will result in an error

 72%|███████▏  | 41/57 [00:00<00:00, 401.87it/s]/home/prakash/anaconda3/envs/FastAI/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass max_patches=18 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/prakash/anaconda3/envs/FastAI/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass max_patches=18 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/prakash/anaconda3/envs/FastAI/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass max_patches=18 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/prakash/anaconda3/envs/FastAI/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass max_patches=18 as keyword args. From version 0.25 passing these as positional arguments will result i

In [6]:
gaussian_noise_std = 25
train_images = train_images+np.random.randn(train_images.shape[0], train_images.shape[1], train_images.shape[2])*gaussian_noise_std
val_images = val_images+np.random.randn(val_images.shape[0], val_images.shape[1], val_images.shape[2])*gaussian_noise_std

In [7]:
print("Shape of training images:", train_images.shape, "Shape of validation images:", val_images.shape)
train_images = utils.augment_data(train_images)

Shape of training images: (1026, 128, 128) Shape of validation images: (198, 128, 128)
Raw image size after augmentation (8208, 128, 128)


# Training Data Preparation

For training we need to follow some preprocessing steps first which will prepare the data for training purposes.

### Data preprocessing
We first divide the data into training and validation sets with 85% images allocated to training set  and rest to validation set. Then we augment the training data 8-fold by 90 degree rotations and flips.

We extract overlapping patches of size ```patch_size x patch_size``` from training and validation images. Specify the parameter ```patch_size```. The number of patches to be extracted is automatically determined depending on the size of images.

Finally, we compute the mean and standard deviation of our combined train and validation sets and do some additional preprocessing.

In [8]:
data_mean, data_std = utils.getMeanStdData(train_images, val_images)

x_train, x_val = utils.convertToFloat32(train_images, val_images)
x_train_extra_axis = x_train[:,np.newaxis]
x_val_extra_axis = x_val[:,np.newaxis]

x_train_tensor = utils.convertNumpyToTensor(x_train_extra_axis)
x_val_tensor = utils.convertNumpyToTensor(x_val_extra_axis)

In [9]:
print("Shape of training tensor:", x_train_tensor.shape)

Shape of training tensor: torch.Size([8208, 1, 128, 128])


# Configure DivNoising model

Here we specify some parameters of our DivNoising network needed for training. <br> 
The parameter <code>z_dim</code> specifies the size of the bottleneck dimension corresponding to each pixel. <br> 
The parameter <code>in_channels</code> specifies the number of input channels which for this dataset is 1.<br> 
We currently have support for only 1 channel input but it may be extended to arbitrary number of channels in the future. <br>
The parameter <code>init_filters</code> specifies the number of filters in the first layer of the network. <br>
The parameter <code>n_depth</code> specifies the depth of the network. 
The parameter <code>batch_size</code> specifies the batch size used for training. <br>
The parameter <code>n_filters_per_depth</code> specifies the number of convolutions per depth. <br>
The parameter <code>directory_path</code> specifies the directory where the model will be saved. <br>
The parameter <code>n_epochs</code> specifies the number of training epochs. <br>
The parameter <code>lr</code> specifies the learning rate. <br>
The parameter <code>val_loss_patience</code> specifies the number of epochs after which training will be terminated if the validation loss does not decrease by a factor of 1e-6. <br>
The parameter <code>noiseModel</code> is the noise model you want to use. Run the notebook  ```Convallaria-CreateNoiseModel.ipynb```, if you have not yet generated the noise model for this dataset yet. If set to None a Gaussian noise model is used.<br>
The parameter <code>gaussian_noise_std</code> is the standard deviation of the Gaussian noise model. This should only be set if 'noiseModel' is None. Otherwise, if you have created a noise model already, set it to ```None```. <br>
The parameter <code>model_name</code> specifies the name of the model with which the weights will be saved for prediction later.

__Note:__ We observed good performance of the DivNosing network for most datasets with the default settings in the next cell. However, we also observed that playing with the paramaters sensibly can also improve performance.

In [10]:
in_channels = 1
init_filters = 10
batch_size=32
directory_path = "./"
n_epochs = int(22000000/(x_train_tensor.shape[0])) # A heurisitc to set the number of epochs
lr=0.001
val_loss_patience = 100
gaussian_noise_std = gaussian_noise_std
noise_model_params= None
noiseModel = None
model_name = "bsd68-10filters-blur"

# Train network

__Note:__ We observed that for certain datasets, the KL loss goes towards 0. This phenomenon is called ```posterior collapse``` and is undesirable.
We prevent it by aborting and restarting the training once the KL dropy below a threshold (```kl_min```).
An alternative approach is a technique called *KL Annealing* where we increase the weight on KL divergence loss term from 0 to 1 gradually in a numer of steps.
This cann be activated by setting the parameter ```kl_annealing``` to ```True```. <br>
The parameter ```kl_start``` specifies the epoch when KL annelaing will start. <br>
The parameter ```kl_annealtime``` specifies until which epoch KL annealing will be operational. <br>
If the parameter ```kl_annealing``` is set to ```False```, the values of ```kl_start``` and  ```kl_annealtime``` are ignored.

In [11]:
train_dataset = dataLoader.MyDataset(x_train_tensor,x_train_tensor)
val_dataset = dataLoader.MyDataset(x_val_tensor,x_val_tensor)

trainHist, reconHist, klHist, valHist = None, None, None, None
attempts=0
while trainHist is None:
    attempts+=1
    print('start training: attempt '+ str(attempts))
    vae = model_multi.MultiVAE(in_channels=in_channels,start_filts=init_filters,depth=3,blur=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    trainHist, reconHist, klHist, valHist = training_multi.trainNetwork(net=vae, train_loader=train_loader, 
                                                                     val_loader=val_loader,
                                                                     device=device,directory_path=directory_path,
                                                                     model_name=model_name,
                                                                     n_epochs=n_epochs, batch_size=batch_size,lr=lr,
                                                                     val_loss_patience = val_loss_patience,
                                                                     kl_annealing = False,
                                                                     kl_start = 0, 
                                                                     kl_annealtime = 3,
                                                                     kl_min=1e-5,
                                                                     data_mean =data_mean,data_std=data_std, 
                                                                     noiseModel = noiseModel,
                                                                     gaussian_noise_std = gaussian_noise_std)
    

start training: attempt 1
Epoch[1/2680] Training Loss: 1.167 Reconstruction Loss: 1.007 KL Loss: 0.160
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
saving ./bsd68-10filters-blurbest_vae.net
Patience: 0 Validation Loss: 0.9102156758308411 Min validation loss: 0.9102156758308411
Time for epoch: 22seconds
Est remaining time: 16:22:18 or 58938 seconds
----------------------------------------
Epoch[2/2680] Training Loss: 0.837 Reconstruction Loss: 0.655 KL Loss: 0.183
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
saving ./bsd68-10filters-blurbest_vae.net
Patience: 0 Validation Loss: 0.8324646353721619 Min validation loss: 0.8324646353721619
Time for epoch: 23seconds
Est remaining time: 17:06:34 or 61594 seconds
----------------------------------------
Epoch[3/2680] Training Loss: 0.777 Reconstruction Loss: 0.574 KL Loss: 0.203
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
saving ./bsd68-10filters-blurbest_vae.net
Patience: 0 Validation Loss: 0.7737792730331

saving ./bsd68-10filters-blurbest_vae.net
Patience: 0 Validation Loss: 0.6152826547622681 Min validation loss: 0.6152826547622681
Time for epoch: 23seconds
Est remaining time: 16:58:31 or 61111 seconds
----------------------------------------
Epoch[24/2680] Training Loss: 0.606 Reconstruction Loss: 0.333 KL Loss: 0.274
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 1 Validation Loss: 0.6170469522476196 Min validation loss: 0.6152826547622681
Time for epoch: 23seconds
Est remaining time: 16:58:08 or 61088 seconds
----------------------------------------
Epoch[25/2680] Training Loss: 0.605 Reconstruction Loss: 0.331 KL Loss: 0.274
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
saving ./bsd68-10filters-blurbest_vae.net
Patience: 0 Validation Loss: 0.6139945387840271 Min validation loss: 0.6139945387840271
Time for epoch: 23seconds
Est remaining time: 16:57:45 or 61065 seconds
----------------------------------------
Epoch[26/2680] Training Loss: 0.604 Reconst

Epoch[47/2680] Training Loss: 0.594 Reconstruction Loss: 0.313 KL Loss: 0.281
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 2 Validation Loss: 0.6089763045310974 Min validation loss: 0.5975126028060913
Time for epoch: 23seconds
Est remaining time: 16:49:19 or 60559 seconds
----------------------------------------
Epoch[48/2680] Training Loss: 0.592 Reconstruction Loss: 0.311 KL Loss: 0.281
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 3 Validation Loss: 0.6001708507537842 Min validation loss: 0.5975126028060913
Time for epoch: 23seconds
Est remaining time: 16:48:56 or 60536 seconds
----------------------------------------
Epoch[49/2680] Training Loss: 0.592 Reconstruction Loss: 0.311 KL Loss: 0.281
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 4 Validation Loss: 0.6131529211997986 Min validation loss: 0.5975126028060913
Time for epoch: 23seconds
Est remaining time: 16:48:33 or 60513 seconds
---------------------------------

Patience: 7 Validation Loss: 0.5996329188346863 Min validation loss: 0.5955206155776978
Time for epoch: 23seconds
Est remaining time: 16:40:07 or 60007 seconds
----------------------------------------
Epoch[72/2680] Training Loss: 0.587 Reconstruction Loss: 0.305 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 8 Validation Loss: 0.5985516309738159 Min validation loss: 0.5955206155776978
Time for epoch: 23seconds
Est remaining time: 16:39:44 or 59984 seconds
----------------------------------------
Epoch[73/2680] Training Loss: 0.588 Reconstruction Loss: 0.305 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 9 Validation Loss: 0.6018268465995789 Min validation loss: 0.5955206155776978
Time for epoch: 23seconds
Est remaining time: 16:39:21 or 59961 seconds
----------------------------------------
Epoch[74/2680] Training Loss: 0.586 Reconstruction Loss: 0.304 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_

Epoch[96/2680] Training Loss: 0.584 Reconstruction Loss: 0.302 KL Loss: 0.283
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 1 Validation Loss: 0.6011563539505005 Min validation loss: 0.5884145498275757
Time for epoch: 23seconds
Est remaining time: 16:30:32 or 59432 seconds
----------------------------------------
Epoch[97/2680] Training Loss: 0.585 Reconstruction Loss: 0.302 KL Loss: 0.283
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 2 Validation Loss: 0.5941202044487 Min validation loss: 0.5884145498275757
Time for epoch: 23seconds
Est remaining time: 16:30:09 or 59409 seconds
----------------------------------------
Epoch[98/2680] Training Loss: 0.584 Reconstruction Loss: 0.301 KL Loss: 0.283
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 3 Validation Loss: 0.6088322997093201 Min validation loss: 0.5884145498275757
Time for epoch: 24seconds
Est remaining time: 17:12:48 or 61968 seconds
------------------------------------

...

Patience: 25 Validation Loss: 0.5936107039451599 Min validation loss: 0.5819515585899353
Time for epoch: 23seconds
Est remaining time: 15:34:34 or 56074 seconds
----------------------------------------
Epoch[243/2680] Training Loss: 0.577 Reconstruction Loss: 0.295 KL Loss: 0.283
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 26 Validation Loss: 0.5865060687065125 Min validation loss: 0.5819515585899353
Time for epoch: 23seconds
Est remaining time: 15:34:11 or 56051 seconds
----------------------------------------
Epoch[244/2680] Training Loss: 0.577 Reconstruction Loss: 0.295 KL Loss: 0.283
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 27 Validation Loss: 0.588179349899292 Min validation loss: 0.5819515585899353
Time for epoch: 23seconds
Est remaining time: 15:33:48 or 56028 seconds
----------------------------------------
Epoch[245/2680] Training Loss: 0.577 Reconstruction Loss: 0.294 KL Loss: 0.283
kl_weight: 1.0
saving ./bsd68-10filters-blur

Epoch[267/2680] Training Loss: 0.577 Reconstruction Loss: 0.295 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 21 Validation Loss: 0.5872944593429565 Min validation loss: 0.5814699530601501
Time for epoch: 23seconds
Est remaining time: 15:24:59 or 55499 seconds
----------------------------------------
Epoch[268/2680] Training Loss: 0.577 Reconstruction Loss: 0.294 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 22 Validation Loss: 0.5852535963058472 Min validation loss: 0.5814699530601501
Time for epoch: 23seconds
Est remaining time: 15:24:36 or 55476 seconds
----------------------------------------
Epoch[269/2680] Training Loss: 0.577 Reconstruction Loss: 0.295 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 23 Validation Loss: 0.588620662689209 Min validation loss: 0.5814699530601501
Time for epoch: 23seconds
Est remaining time: 15:24:13 or 55453 seconds
----------------------------

Patience: 45 Validation Loss: 0.5872883200645447 Min validation loss: 0.5814699530601501
Time for epoch: 23seconds
Est remaining time: 15:15:47 or 54947 seconds
----------------------------------------
Epoch[292/2680] Training Loss: 0.576 Reconstruction Loss: 0.294 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 46 Validation Loss: 0.5830668210983276 Min validation loss: 0.5814699530601501
Time for epoch: 23seconds
Est remaining time: 15:15:24 or 54924 seconds
----------------------------------------
Epoch[293/2680] Training Loss: 0.576 Reconstruction Loss: 0.293 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 47 Validation Loss: 0.5888060927391052 Min validation loss: 0.5814699530601501
Time for epoch: 23seconds
Est remaining time: 15:15:01 or 54901 seconds
----------------------------------------
Epoch[294/2680] Training Loss: 0.576 Reconstruction Loss: 0.294 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blu

Epoch[316/2680] Training Loss: 0.576 Reconstruction Loss: 0.293 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 70 Validation Loss: 0.587335467338562 Min validation loss: 0.5814699530601501
Time for epoch: 23seconds
Est remaining time: 15:06:12 or 54372 seconds
----------------------------------------
Epoch[317/2680] Training Loss: 0.576 Reconstruction Loss: 0.293 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 71 Validation Loss: 0.5877234935760498 Min validation loss: 0.5814699530601501
Time for epoch: 23seconds
Est remaining time: 15:05:49 or 54349 seconds
----------------------------------------
Epoch[318/2680] Training Loss: 0.576 Reconstruction Loss: 0.293 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 72 Validation Loss: 0.583625316619873 Min validation loss: 0.5814699530601501
Time for epoch: 23seconds
Est remaining time: 15:05:26 or 54326 seconds
-----------------------------

Patience: 94 Validation Loss: 0.591560959815979 Min validation loss: 0.5814699530601501
Time for epoch: 23seconds
Est remaining time: 14:57:00 or 53820 seconds
----------------------------------------
Epoch[341/2680] Training Loss: 0.575 Reconstruction Loss: 0.293 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 95 Validation Loss: 0.5883181691169739 Min validation loss: 0.5814699530601501
Time for epoch: 23seconds
Est remaining time: 14:56:37 or 53797 seconds
----------------------------------------
Epoch[342/2680] Training Loss: 0.575 Reconstruction Loss: 0.293 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blurlast_vae.net
Patience: 96 Validation Loss: 0.5870451927185059 Min validation loss: 0.5814699530601501
Time for epoch: 23seconds
Est remaining time: 14:56:14 or 53774 seconds
----------------------------------------
Epoch[343/2680] Training Loss: 0.575 Reconstruction Loss: 0.293 KL Loss: 0.282
kl_weight: 1.0
saving ./bsd68-10filters-blur

# Plotting losses

In [ ]:
trainHist=np.load(directory_path+"/train_loss.npy")
reconHist=np.load(directory_path+"/train_reco_loss.npy")
klHist=np.load(directory_path+"/train_kl_loss.npy")
valHist=np.load(directory_path+"/val_loss.npy")

In [ ]:
plt.figure(figsize=(18, 3))
plt.subplot(1,3,1)
plt.plot(trainHist,label='training')
plt.plot(valHist,label='validation')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend()

plt.subplot(1,3,2)
plt.plot(reconHist,label='training')
plt.xlabel("epochs")
plt.ylabel("reconstruction loss")
plt.legend()

plt.subplot(1,3,3)
plt.plot(klHist,label='training')
plt.xlabel("epochs")
plt.ylabel("KL loss")
plt.legend()
plt.show()